# Proyecto Bimestral: Implementación de un Sistema RAG (Retrieval-Augmented Generation)
## 1. Introducción
Este proyecto consiste en diseñar e implementar un sistema RAG (Retrieval-Augmented Generation). Se busca combinar técnicas de Recuperación de Información (RI) con modelos de generación de texto, permitiendo que un modelo genere respuestas a partir de documentos relevantes recuperados desde  un corpus.

## 2. Objetivo del Proyecto
Desarrollar un sistema RAG que:
1. Recupere documentos relevantes a partir de una consulta del usuario utilizando técnicas de RI.
2. Genere respuestas basadas en los documentos recuperados utilizando un modelo de lenguaje avanzado.

## 3. Fases del pre-procesamiento de la data

### 3.1. Librerias necesarias

El código realiza las siguientes operaciones con la librería `pandas`, una herramienta popular en Python para la manipulación y análisis de datos:


In [ ]:
import pandas as pd

# Obtener el nombre del archivo subido
file_name = "/content/Entrevista_LuisaGonzales_JuanCueva.xlsx"

# Cargar el archivo en un DataFrame
df = pd.read_excel(file_name, engine='openpyxl')

df

,ID,Candidato,Temas,Descripción,Entrevista
0,LI1,Leonidas Iza,"política electoral en Ecuador, el papel del m...","Es una entrevista con Leonidas Iza, líder indí...",Pero es un gusto compartir esta mañana con el...
1,LI2,Leonidas Iza,"plurinacionalidad, crítica al sistema, progres...",Leonidas Iza critica la falta de participación...,Yo estuve vinculado en la iglesia católica du...
2,LI3,Leonidas Iza,"Problemas estructurales y sus consecuencias, D...",presenta un diagnóstico sobre la crisis económ...,PLAN DE TRABAJO\nEl abrazo del pueblo\nI. DIAG...
3,LT1,Luis Tillería,"políticas fiscales, legalización de sustancias...",El texto abarca temas como la crítica a las po...,¿Tú demás quieres hacernos un paraíso fiscal?...
4,LT2,Luis Tillería,"idea de la libertad, diferencia entre liberta...","Luis Felipe Tijería, candidato presidencial, c...",¿Qué tal mis amigos? Bienvenidos a un nuevo p...
5,LT3,Luis Tillería,"declive económico de Ecuador, comparación con ...",Luis Felipe Tijería reflexiona sobre el decliv...,¿Por qué el colegio es el país más rico de La...
6,HK1,Henry Kronfle,"Indecisión política, desconfianza en los polít...",La discusión aborda la falta de decisión de gr...,Y el domingo de debate. El domingo del debate...
7,HK2,Henry Kronfle,"Confianza en principios y valores, críticas al...",La conversación gira en torno a la trayectoria...,A mí también me han saltado con pistones en l...
8,HK3,Henry Kronfle,"Seguridad en Ecuador, derechos humanos, políti...","En este segmento, el candidato Henry Kronfle ...","Hola, bienvenidos al segmento electoral de Es..."



El método `info()` de un DataFrame de `pandas` se utiliza para obtener un resumen conciso de un DataFrame. Aquí te explico qué hace específicamente este método y cómo se usa en el contexto del código proporcionado:

```python
df.info()


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           9 non-null      object
 1   Candidato    9 non-null      object
 2   Temas        9 non-null      object
 3   Descripción  9 non-null      object
 4   Entrevista   9 non-null      object
dtypes: object(5)
memory usage: 492.0+ bytes


### 3.2. Procesamiento de Texto con spaCy y NLTK

El código presentado realiza una serie de tareas de procesamiento de texto utilizando las librerías `spaCy` y `NLTK`, ambas ampliamente utilizadas en el procesamiento del lenguaje natural (NLP). A continuación, se detalla el propósito y funcionamiento de cada segmento del código:

In [ ]:
!python -m spacy download es_core_news_sm
import re
import spacy
import nltk
from nltk.corpus import stopwords

# Descargar stopwords en español
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

# Cargar modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

def limpiar_texto(texto):
    """ Normaliza, elimina caracteres especiales y convierte a minúsculas """
    texto = texto.lower()  # Minúsculas
    texto = re.sub(r'\s+', ' ', texto)  # Espacios extra
    texto = re.sub(r'[^\w\s]', '', texto)  # Caracteres especiales
    return texto.strip()

def procesar_texto(texto):
    """ Aplica limpieza, eliminación de stopwords y lematización """
    texto = limpiar_texto(texto)
    doc = nlp(texto)
    palabras_limpias = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]
    return " ".join(palabras_limpias)

# Aplicar procesamiento a las columnas
for columna in ["Temas", "Descripción", "Entrevista"]:
    df[columna] = df[columna].astype(str).apply(procesar_texto)

df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 70.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,ID,Candidato,Temas,Descripción,Entrevista
0,LI1,Leonidas Iza,política electoral ecuador papel movimiento in...,entrevista leonida izar líder indígén político...,gusto compartir mañana ingeniero leonida izo b...
1,LI2,Leonidas Iza,plurinacionalidad crítico sistema progresismo ...,leonida iza criticar falta participación ciuda...,vinculado iglesia católico 10 año oir moquito ...
2,LI3,Leonidas Iza,problema estructural consecuencia desigualdad ...,presentar diagnóstico crisis económico social ...,plan trabajo abrazo pueblo i diagnóstico ecuad...
3,LT1,Luis Tillería,política fiscal legalización sustancia petróle...,texto abarca tema crítica política fiscal refl...,demá quieres hacer yo paraíso fiscal bueno mej...
4,LT2,Luis Tillería,idea libertad diferenciar libertario liberal c...,luis felipe tijería candidato presidencial com...,tal amigo bienvenido nuevo podcast verdad nuev...
5,LT3,Luis Tillería,declivir económico ecuador comparación chile m...,luis felipe tijería reflexión declive económic...,colegio país rico latinoamérica siempre certez...
6,HK1,Henry Kronfle,indecisión político desconfianza político búsq...,discusión abordo falta decisión gran parte ele...,domingo debate domingo debate así cómo prepará...
7,HK2,Henry Kronfle,confianza principio valor crítico sistema polí...,conversación gira torno trayectoria político é...,saltar pistón cabeza mejor amiga escuela espos...
8,HK3,Henry Kronfle,seguridad ecuador derechos humanos política se...,segmento candidato henry kronfle aborda tema c...,holar bienvenido segmento electoral político p...


### 3.3. Guardar y Descargar un Archivo CSV en Google Colab

El código proporcionado muestra cómo guardar un DataFrame de `pandas` en un archivo CSV y luego descargarlo directamente desde un entorno de Google Colab. Este procedimiento es especialmente útil para manipular y exportar datos en entornos de computación en la nube. A continuación se detalla cada parte del código:


In [ ]:
from google.colab import files

# Guardar el DataFrame como CSV
df.to_csv("datos_procesados.csv", index=False, encoding='utf-8')

# Descargar el archivo
files.download("datos_procesados.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>